#Create Agents to Research a company for interview prep and generate a research report for the candidate

Using CrewAI framework build a multi-agent systems and that will research a company and generate useful & relevant
interview preparation material such as Company profile and sample Q&A .

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, we'll be using OpenAI's `gpt-4o`.

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Executive Coach

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

### Agent: Executive Interview Coach

In [ ]:
executive_coach = Agent(
    role="Executive Coach Specializing in Interview Preparation",
    goal="Coach job seeker to get hired as Chief Technology Officer (CTO) at {company}",
    backstory="You're a seasoned and experienced executive coach that is coaching a job seeker to prepare for a CTO interview at {company}. "
              "You provide expert advice and strategies to help them excel in the interview."
              "You train them to constantly link technology choices to P&L (Profit and Loss). "
              "You help them differentiate between humility and insecurity. "
              "You understand the current technology landscape and its impact on business strategy that is relevant to {company}. "
              "You know that C-level interviews are 50% about competence and 50% about confidence"
              "Your work is the basis for "
              "the Interview Researcher to research and put together a comprehensive guide for interview preparation at {company}.",
    allow_delegation=False,
	verbose=True
)

### Agent: Company researcher

In [ ]:
interview_researcher = Agent(
    role="Interview Researcher",
    goal="Conduct in-depth research on {company} using the interview preparation guide created by executive coach.",
    backstory="You're working on researching {company} to prepare a comprehensive guide for a CTO job interview. "
              "You base your research on various sources to compile a detailed profile of the company, "
              "including its product and services, operations, financials, tech stack, leadership team, and industry landscape. "
              "You use various sources such as the company's website, news articles, press releases, industry reports, and social media profiles to gather relevant information. "
              "You analyze the information to identify key trends, challenges, and opportunities that may impact the company's operations and growth prospects. "
              "Your work is essential for the Mock Interviewer to provide tailored interview questions and sample answers to the job seeker."
              "You collaborate with the Executive Coach, who provides an outline and context for your research. "
              "and you ensure that your research aligns with the coaching strategies and objectives set by the Executive Coach."
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Executive Coach. "
              "You ensure that your research is comprehensive, accurate, and relevant to the job seeker's interview preparation needs." ,
    allow_delegation=False,
    verbose=True
)

### Agent: Mock Interviewer

    "You leverage the comprehensive research report provided by the Interview Researcher to tailor the interview preparation. "
    

In [ ]:
interviewer = Agent(
    role="Mock Interviewer",
    goal="Conduct mock interviews for the CTO positions with the job seeker based on the research and coaching provided.",
    backstory="You are an expert interviewer who conducts mock interviews for the CTO position at {company} with the job seeker. "
              "You use the research and insights provided by the Interview Researcher and Executive Coach "
              "to create realistic interview scenarios and questions. "
              "Your goal is to make sure the job seeker is well-prepared for the actual interview. "
              "You possess deep expertise and Technical Knowledge of the strategic implications of technology to achieve business outcomes. "
              "You understand the KPIs that matter to the C-suite: CAC (Customer Acquisition Cost), "
              "LTV (Lifetime Value), ROI, and Time-to-Market. You ensure your questions and answers target to these. "
              "You are knowledgeable about CTO Archetypes and tailor your questions accordingly. "
              "You provide constructive feedback to help the job seeker improve their performance and confidence. "
              "You are knowledgeable about the latest trends and challenges in the tech industry, and you incorporate this knowledge into your mock interviews. "
              "You collaborate closely with the Executive Coach and Interview Researcher to ensure a cohesive and effective interview preparation process. ",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Create Coach Plan
Detailed, comprehensive and relevant interview preparation guide for CTO

In [ ]:
executive_coaching_task = Task(
    description=(
        "1. Create personal brand and positioning for CTO role at {company}. "
            "For example, “scaleup SaaS CTO, strong in cloud-native architecture and building senior engineering leadership”.\n"
        "2. Leadership and executive presence coaching: "
            "Help the job seeker understand their leadership style and how to effectively communicate their vision and strategy. "
            "This includes speaking to the board, storytelling with data, managing conflict with peers, and handling underperformance at VP/Director level. "
            "communication to non-technical stakeholders: explaining trade-offs, risk, ROI, and technology strategy in concise, business-first language.\n"
        "3. candidate describes their current architecture, technical strategy, and roadmap so it sounds coherent, robust, and business-aligned.\n"
        "4. Prepare answers to common CTO interview questions, including technical questions, leadership questions, "
            "and situational questions.\n"
        "5. Conduct mock interviews to practice answering questions and receiving feedback.\n"
        "6. Identify 3 to 5“signature” initiatives with quantifiable impact (e.g., uptime, revenue, margins, velocity, cost reduction, security posture) and weave them into interview answers."
    ),
    expected_output="A detailed, comprehensive and relevant interview preparation guide for someone interviewing for a Chief Technology Officer position.",
    agent=executive_coach,
)

### Task: Company research for interview preparation

In [ ]:
interview_research_task = Task(
    description=(
        "1. Use the coaching plan to do a comprehensive in-depth research on {company}.\n"
		"2. Sections/Subtitles are properly named that are relevant for the interview preparation process"
        "3. Ensure the research report is structured to assist with interview preparation, "
            "gather insights, examples, and data points relevant to the CTO role at {company}.\n"
        "4. Ensure research report will help the job seeker understand the company's technology landscape, "
            "and competitive positioning.\n"
        "5. Include relevant data, sources, and references to help prepare a 30-60-90 day plan.\n"
        "6. Use markdown format for the report.\n"
        "7. Each section should have 3 or 4 paragraphs.\n"
    ),
    expected_output="A very exhaustive and detailed research report "
        "in markdown format, very helpful for the interviewer to prepare the interview "
        "sample questions and asnwers and the various rounds of interviews with the CEO, Board, Senior Vice Presidents "
        "and each section should have 3 or 4 paragraphs.",
    agent=interview_researcher,
)

### Task: Create detail and comprehensive interview preparation material for a CTO position at a given company

In [ ]:
mock_interview_task = Task(
    description=("Leveraging the research and coaching provided, conduct a mock interview for the CTO position at {company} "
                 "with the job seeker."),
    expected_output="A well-structured mock interview session that covers key topics "
                    "relevant to the CTO role at {company}. "
                    "For each section in the company research report, create 10 interview questions and provide detailed answers.",
    agent=interviewer
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [ ]:
crew = Crew(
    agents=[executive_coach, interview_researcher, interviewer],
    tasks=[executive_coaching_task, interview_research_task, mock_interview_task],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"company":"MSCI Inc."})

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).